In [1]:
import sqlite3
DB_FILE = "ql_nhan_vien.db"

def connect_db(db_file=DB_FILE):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = ON;") 
    return conn, cursor

def create_tables():
    conn, cursor = connect_db()
    try:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS PHONG (
                Id INTEGER PRIMARY KEY,
                Name TEXT NOT NULL,
                Price REAL NOT NULL,
                Amount INTEGER NOT NULL
            );
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS NHAN_VIEN (
                id INTEGER PRIMARY KEY,
                ho_ten TEXT NOT NULL,
                tuoi INTEGER NOT NULL,
                dia_chi TEXT NOT NULL,
                luong REAL NOT NULL,
                Id_phong INTEGER, 
                
                -- Định nghĩa Khóa ngoại tham chiếu đến PHONG(Id)
                FOREIGN KEY (Id_phong) REFERENCES PHONG(Id)
            );
        """)
        conn.commit()
        print(f"✅ Đã tạo CSDL '{DB_FILE}' và hai bảng PHONG/NHAN_VIEN với Khóa ngoại.")
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi tạo bảng: {e}")
    finally:
        conn.close()
def add_departments():
    """Thêm dữ liệu mẫu vào bảng PHONG."""
    conn, cursor = connect_db()
    try:
        departments = [
            (1, 'Phòng Kỹ Thuật', 1000.0, 5),     
            (2, 'Phòng Kinh Doanh', 500.0, 10),    
            (3, 'Phòng Hành Chính', 200.0, 3)     
        ]
        # Thêm dữ liệu vào các cột Id, Name, Price, Amount
        cursor.executemany(
            "INSERT OR IGNORE INTO PHONG (Id, Name, Price, Amount) VALUES (?, ?, ?, ?)", 
            departments
        )
        conn.commit()
        print(f"➕ Đã thêm {cursor.rowcount} phòng ban mẫu vào bảng PHONG.")
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi thêm phòng ban: {e}")
    finally:
        conn.close()

def add_employees():
    """Thêm dữ liệu mẫu vào bảng NHAN_VIEN."""
    conn, cursor = connect_db()
    try:
        employees = [
            # Thuộc Phòng Kỹ Thuật (ID=1)
            ('Nguyễn Văn A', 25, 'Hà Nội', 15000000.0, 1),
            # Thuộc Phòng Kinh Doanh (ID=2)
            ('Trần Thị B', 30, 'TP HCM', 22000000.5, 2),('Lê Văn C', 22, 'Đà Nẵng', 12000000.0, 2),
            # Thuộc Phòng Hành Chính (ID=3)
            ('Phạm Văn D', 45, 'Hà Nội', 18500000.0, 3),
        ]
        
        cursor.executemany(
            "INSERT INTO NHAN_VIEN (ho_ten, tuoi, dia_chi, luong, Id_phong) VALUES (?, ?, ?, ?, ?)",
            employees
        )
        conn.commit()
        print(f"➕ Đã thêm {cursor.rowcount} nhân viên mẫu vào bảng NHAN_VIEN.")
    except sqlite3.IntegrityError as e:
        print(f"❌ Lỗi Khóa ngoại (Id_phong không tồn tại): {e}")
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi thêm nhân viên: {e}")
    finally:
        conn.close()

def display_employees():
    """Hiển thị danh sách nhân viên cùng tên phòng ban."""
    conn, cursor = connect_db()
    try:
        sql = """
            SELECT 
                NV.id, NV.ho_ten, NV.tuoi, NV.luong, P.Name 
            FROM NHAN_VIEN NV
            JOIN PHONG P ON NV.Id_phong = P.Id
            ORDER BY NV.id;
        """
        cursor.execute(sql)
        employees = cursor.fetchall()

        print("\n--- DANH SÁCH NHÂN VIÊN VÀ PHÒNG BAN ---")
        if not employees:
            print("Không có nhân viên nào.")
            return

        print(f"| {'ID':<4} | {'HỌ TÊN':<20} | {'TUỔI':<5} | {'LƯƠNG':<15} | {'PHÒNG BAN':<20} |")
        print("-" * 70)
        for emp in employees:
            luong_format = f"{emp[3]:,.0f}" # Định dạng số
            print(f"| {emp[0]:<4} | {emp[1]:<20} | {emp[2]:<5} | {luong_format:<15} | {emp[4]:<20} |")
        print("-" * 70)
        
    except sqlite3.Error as e:
        print(f"❌ Lỗi khi hiển thị nhân viên: {e}")
    finally:
        conn.close()

# ----------------------------------------------------
# KHỞI CHẠY VÀ DEMO ỨNG DỤNG QUẢN LÝ NHÂN VIÊN
# ----------------------------------------------------

def run_employee_management_app():
    # 1. Khởi tạo cấu trúc CSDL
    create_tables()
    
    # 2. Thêm dữ liệu phòng ban 
    add_departments()
    
    # 3. Thêm dữ liệu nhân viên
    add_employees()
    
    # 4. Hiển thị kết quả
    display_employees()

# Chạy DEMO
run_employee_management_app()

✅ Đã tạo CSDL 'ql_nhan_vien.db' và hai bảng PHONG/NHAN_VIEN với Khóa ngoại.
➕ Đã thêm 3 phòng ban mẫu vào bảng PHONG.
➕ Đã thêm 4 nhân viên mẫu vào bảng NHAN_VIEN.

--- DANH SÁCH NHÂN VIÊN VÀ PHÒNG BAN ---
| ID   | HỌ TÊN               | TUỔI  | LƯƠNG           | PHÒNG BAN            |
----------------------------------------------------------------------
| 1    | Nguyễn Văn A         | 25    | 15,000,000      | Phòng Kỹ Thuật       |
| 2    | Trần Thị B           | 30    | 22,000,000      | Phòng Kinh Doanh     |
| 3    | Lê Văn C             | 22    | 12,000,000      | Phòng Kinh Doanh     |
| 4    | Phạm Văn D           | 45    | 18,500,000      | Phòng Hành Chính     |
----------------------------------------------------------------------
